# Introduction

`icpmsflow` is a collection of routines to analyze icpms data.   

The main component of icpms flow is the `ICPMSAnalysis` class.  This class handles standard manipulations of icpms data.  Below, we show an abbreviated use case.


## main modules

In [15]:
import icpmsflow
# for finding paths to files
from pathlib import Path

## read in data

In [16]:
# In this directory, there are files '001SMPL-*.csv'
# get list of paths
paths = list(Path('.').glob('00*.csv'))
print(paths)

[PosixPath('001SMPL-4.csv'), PosixPath('001SMPL-5.csv'), PosixPath('001SMPL-7.csv'), PosixPath('001SMPL-6.csv'), PosixPath('001SMPL-2.csv'), PosixPath('001SMPL-3.csv'), PosixPath('001SMPL-1.csv'), PosixPath('001SMPL-10.csv'), PosixPath('001SMPL-11.csv'), PosixPath('001SMPL-13.csv'), PosixPath('001SMPL-12.csv'), PosixPath('001SMPL-8.csv'), PosixPath('001SMPL-9.csv')]


## Create ICPMSAnalaysis object from these paths

In [17]:
ds = icpmsflow.ICPMSAnalysis.from_paths(paths)

## working with data

There are two components of the `ICPMSAnalysis` class.  The icpms data, and the bounds_data, which contains the baseline and signal regions.  Right now, these regions are per batch.  However, this can be adjusted down the road.  
You can set the `bounds_data` manually, or set it with the `ds.add_bounds` method, shown below.  This method takes the derivative of the signal, and looks for the minimum/maximum peaks to find the jumps in the data.  Here we use the optional parameters

    * kernel_size : apply median filter with this kernel size.  Useful to smooth out noisy data
    * z_threshold : z score applied to extrema.  Only consider element min/max where scipy.stats.zscore is less than 
    This cutoff
    * shift : baseline = (tmin, max - shift[0]), signal = (max+shift[1], min - shift[2])
    
    
We also apply `snap_bounds_minmax` to ensure that the bounds are withing the time min/max of the data
    

In [19]:
# add in bounds_data
ds = (
    ds
    .add_bounds(kernel_size=21, z_threshold=2, shift=(5, 10, 2))
    .snap_bounds_minmax()
)


# new functionality to work with indexes

In [22]:
bounds_data = ds.bounds_data.reset_index()
bounds_data.head()

,batch,type_bound,lower_bound,upper_bound
0,1FEB21610CAL.b,baseline,2.9896,2.9896
1,1FEB21610CAL.b,signal,10.0000,10.0000
2,1FEB21610CAL2.b,baseline,2.9896,18.8469
3,1FEB21610CAL2.b,signal,33.8469,105.2727
4,1FEB21610CALEND.b,baseline,2.9885,30.7637


In [25]:
# Note that bounds_data does not have the required multiindex.  
# pass it in with the `set_bounds` method
ds2 = ds.set_bounds(bounds_data)
# this will auto check index and do what need to be done
ds2.bounds_data.head()

lower_bound  upper_bound
batch             type_bound                          
1FEB21610CAL.b    baseline         2.9896       2.9896
                  signal          10.0000      10.0000
1FEB21610CAL2.b   baseline         2.9896      18.8469
                  signal          33.8469     105.2727
1FEB21610CALEND.b baseline         2.9885      30.7637

In [26]:
# to get a copy of the 'bounds_data' without multiindex, use get_bounds() method
ds2.get_bounds()

,batch,type_bound,lower_bound,upper_bound
0,1FEB21610CAL.b,baseline,2.989600,2.989600
1,1FEB21610CAL.b,signal,10.000000,10.000000
2,1FEB21610CAL2.b,baseline,2.989600,18.846900
3,1FEB21610CAL2.b,signal,33.846900,105.272700
4,1FEB21610CALEND.b,baseline,2.988500,30.763700
5,1FEB21610CALEND.b,signal,45.763700,96.332700
6,1FEB21612610CAL.b,baseline,2.988500,19.667638
7,1FEB21612610CAL.b,signal,34.667638,104.633150
8,1FEB21612CALEND.b,baseline,2.988500,19.564921
9,1FEB21612CALEND.b,signal,34.564921,86.330450


In [29]:
# to get integral at bounds, without multiindex, use reset_index flag
(
    ds
    .interpolate_at_bounds(reset_index=True, as_delta=True)
)

,batch,type_bound,Time [Sec],Li7 -> 7,Mg24 -> 24,Mg25 -> 25,Ca43 -> 43,Ca44 -> 44,Sc45 -> 45,Mn55 -> 55,...,Sb121 -> 121,Sb123 -> 123,Ba137 -> 137,Ba138 -> 138,Tl205 -> 205,Pb206 -> 206,Pb207 -> 207,Pb208 -> 208,Th232 -> 232,U238 -> 238
0,1FEB21610CAL.b,baseline,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1FEB21610CAL.b,signal,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,1FEB21610CAL2.b,baseline,15.857300,0.000000,6.698930e+02,1.191820e+02,2.449100e+01,8.961564e+03,6.055080e+02,3.898912e+03,...,1.734820e+02,8.408250e+01,9.592000e+00,5.156125e+02,5.959200e+01,6.299960e+02,2.181645e+02,4.310130e+02,0.000000e+00,0.000000e+00
3,1FEB21610CAL2.b,signal,71.425800,769394.680961,1.488859e+07,2.209025e+06,1.076490e+07,1.654549e+08,7.115185e+07,1.055465e+08,...,7.781612e+07,6.025326e+07,3.297087e+07,2.216522e+08,2.380121e+07,6.260598e+07,5.803994e+07,1.491283e+08,3.031336e+08,2.962285e+08
4,1FEB21610CALEND.b,baseline,27.775200,0.000000,1.795790e+03,3.969220e+02,1.138815e+02,1.613930e+04,2.248044e+03,4.755666e+04,...,5.756945e+02,5.156335e+02,7.390895e+02,2.162949e+03,3.322425e+03,1.059742e+06,9.408332e+05,2.259676e+06,2.394430e+05,4.485703e+05
5,1FEB21610CALEND.b,signal,50.569000,743723.116247,1.501917e+07,2.211924e+06,1.091749e+07,1.752190e+08,7.046617e+07,1.185497e+08,...,7.669150e+07,5.847077e+07,3.343449e+07,2.283092e+08,2.487822e+07,6.599519e+07,6.080410e+07,1.559658e+08,2.880296e+08,2.888063e+08
6,1FEB21612610CAL.b,baseline,16.679138,0.000000,1.283297e+03,1.310006e+02,1.429827e+02,1.230161e+04,8.843196e+02,4.214157e+03,...,6.344506e+02,4.140541e+02,8.637524e+02,4.431393e+03,3.496643e+03,1.205854e+06,1.071326e+06,2.682700e+06,2.980400e+01,4.469550e+01
7,1FEB21612610CAL.b,signal,69.965512,135919.873915,4.245141e+06,5.989769e+05,2.120975e+07,3.385104e+08,1.247900e+07,2.164496e+07,...,1.702737e+07,1.301978e+07,6.303688e+06,4.104306e+07,1.177702e+07,1.899733e+07,1.706213e+07,4.040309e+07,4.130032e+07,5.020590e+07
8,1FEB21612CALEND.b,baseline,16.576421,0.000000,7.600012e+02,2.039483e+02,2.979550e+01,1.164739e+04,1.352137e+03,4.922838e+03,...,1.741538e+02,1.340935e+02,8.853572e+01,7.860107e+02,1.287925e+03,3.842916e+05,3.528826e+05,8.375488e+05,2.980400e+01,0.000000e+00
9,1FEB21612CALEND.b,signal,51.765529,96710.973717,3.074232e+06,4.144688e+05,1.558258e+07,2.505417e+08,9.210285e+06,1.553157e+07,...,1.128468e+07,8.711366e+06,4.274053e+06,2.814473e+07,9.898319e+06,1.051172e+07,9.573015e+06,2.291171e+07,2.848773e+07,3.290575e+07
